In [ ]:
import numpy as np
import os
import gc
import cv2

from skimage import io
from sys import getsizeof
from matplotlib import pyplot as plt
from itertools import combinations
from numpy.linalg import norm
from scipy import ndimage

In [ ]:
def vis_gt(pair_name_, center=None):
    """
    Visualize a moving image of any pair and shows the vectors from the corresponding ground truth file
    @param: pair_name_ name of the pair to show
    """
    # load the moving image of the pair (1st image)
    im = io.imread(os.path.join(os.getcwd(), "Images", pair_name_+"_1.jpg"))
    im_goal = io.imread(os.path.join(os.getcwd(), "Images", pair_name_+"_2.jpg"))
    fig, ax = plt.subplots(1,3, figsize=(15,5))
    ax[0].imshow(im)
    ax[2].imshow(im_goal)

    # open the ground truth file of the pair
    with open(os.path.join(os.getcwd(), "Ground Truth", "control_points_"+pair_name_+"_1_2.txt"), "r") as f: 
        lines = f.readlines()
        # each line has 4 points, convert to float 
        for line in lines:
            str_ = line[:-1].split(" ")
            p = [float(x) for x in str_]
            ax[0].arrow(p[0], p[1], p[2]-p[0], p[3]-p[1], head_width=80)
    if center:
        ax[0].annotate("X", center, c="green")
    return ax

In [ ]:
vis_gt("A01")

In [ ]:
def get_avg_centers(deltas):
    """
    Getting an estimate for the center of rotation of vektors in space
    @param: deltas list of tuples consisting of (x_original, y_original, x_gold, y_gold, dx, dy)
    """
    cen_xs = []
    cen_ys = []
    
    # Calculate estimated centers for each pair
    combs = combinations(deltas, 2)
    for p1, p2 in combs:
        #print(p1, p2)
        est_center_x = func(p1[0], p2[0], p1[5], p2[5])
        est_center_y = func(p1[1], p2[1], p1[4], p2[4])
        cen_xs.append(est_center_x)
        cen_ys.append(est_center_y)
    
    return np.mean(np.asarray(cen_xs)), np.mean(np.asarray(cen_ys))

def func(p1x, p2x, p1dy, p2dy):
    """does math"""
    dys = p2dy-p1dy
    if dys == 0:
        return (p1x + p2x) / 2
    dxs = p1x-p2x
    est_cen = p1x + (p1dy/dys) * dxs
    
    return est_cen

def get_avg_angle(x_cen, y_cen, deltas, debug=False):
    angles = []
    for el in deltas:
        # Get points
        p1 = np.asarray([el[0], el[1]])
        p2 = np.asarray([el[2], el[3]])
        center = np.asarray([x_cen, y_cen])
        
        # Get distances
        dpp = norm(p2-p1)
        dp1c = norm(center-p1)
        dp2c = norm(center-p2)
        
        # Calculate angle at center, knowing 3 sides
        val = (dp1c**2 + dp2c**2 -dpp**2) / (2 * dp1c * dp2c)
        angle = np.degrees(np.arccos(val))
        angles.append(angle)
        
        if debug:
            print("Points:", p1, p2, center)
            print("Lenghts:", dpp, dp1c, dp2c, val)
            print("Angle:", angle)
    if debug:
        print(np.mean(angles))
    return np.mean(angles)

def get_clockwise(cen_x, cen_y, deltas):
    """
    @param: x_cen x value of pivot point
    @param: y_cen y value of pivot point
    @param: deltas list of tuples consisting of (x_original, y_original, x_gold, y_gold, dx, dy)
    returns: 1 if vectors turn counterclockwise around center point
             -1 if clockwise 
    """
    votes = []
    
    for el in deltas:
        # each vector gives 2 votes
        # 1 based on above/below pivot point
        # 1 based on left/right of pivot point
        
        x_pos, y_pos, _, _, dx, dy = el
        
        # if point is above pivot point..
        if y_pos < cen_y:
            # .. looking right means clockwise
            if dx > 0:
                votes.append(-1)
            # .. looking left means counter clockwise
            else:
                votes.append(1)
        # if point is below ..
        else:
            # .. looking right means counter-clockwise
            if dx > 0:
                votes.append(1)
            # .. looking left means clock wise
            else:
                votes.append(-1)
                
        # if point is left of pivot point..
        if x_pos < cen_x:
            # .. looking up means clockwise
            if dy < 0:
                votes.append(-1)
            # .. looking down means counter clockwise
            else:
                votes.append(1)
        # if point is right of pivot point ..
        else:
            # .. looking up means counter-clockwise
            if dy < 0:
                votes.append(1)
            # .. looking down means clock wise
            else:
                votes.append(-1)
                
    # If the majority of votes is positive (for counter-clockwise)
    if np.sum(np.asarray(votes)) > 0:
        # return value for counter clockwise
        return 1
    # else return value for clockwise
    return -1

In [ ]:
# Get input folder of image pairs 
images_path = os.path.join(os.getcwd(), "Images")
image_names = os.listdir(images_path)

for i in range(0, len(image_names), 2):
    # Get Names of Images
    image_name1 = image_names[i]
    image_name2 = image_names[i+1]
    
    # Get Groundtruth name
    pair_name = image_name1[:3] 
    gt_path = os.path.join(os.getcwd(), "Ground Truth") 
    gt_name = "control_points_" + pair_name + "_1_2.txt"
       
    # Open Ground truth file and write points into the list "deltas"
    # Each element in deltas is a tuple consisting of:
    # x_original, y_original, x_gold, y_gold, dx, dy
    deltas = []
    with open(os.path.join(gt_path, gt_name), "r") as f:       
        lines = f.readlines()
        for line in lines:
            str_ = line[:-1].split(" ")
            p = [float(x) for x in str_]
            deltas.append((p[0], p[1], p[2], p[3], p[2]-p[0], p[3]-p[1]))
            
    # Where Delta Y is close to 0, thats the X value of the center
    # And vice versa
    # With center calculate rotation angle of each vector
    # With center + rotationangle do calc:
    x_cen, y_cen = get_avg_centers(deltas)
    angle = get_avg_angle(x_cen, y_cen, deltas)
    direction = get_clockwise(x_cen, y_cen, deltas)
    # round center values after calculating angle, for more precise angle
    center = (round(x_cen), round(y_cen))
    
    # Load image and rotate
    im1 = io.imread(os.path.join(images_path, image_name1))
    im1 = im1 / 255  
    rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=direction*angle, scale=1)
    im1_rot = cv2.warpAffine(src=im1, M=rotate_matrix, dsize=(im1.shape[0], im1.shape[1]))
    
    # Show image1 with vectors, image2 and image1_rotated
    visualize = True
    if visualize:
        ax = vis_gt(pair_name, center=(x_cen, y_cen)) 
        ax[1].imshow(im1_rot)
        plt.show()
    
    # Save imagepair and new ground truths in their respective new folders
    save_images_and_gts = False
    if save_images_and_gts:
        # Converse image to uint8 for saving 
        im1_rot *= 256
        im1_rot = im1_rot.astype("uint8")
        io.imsave(os.path.join(os.getcwd(), "Images_close_to_gt", image_name1), im1_rot)

        # Load the fixed image and save it in new folder as well, so that pairs are together
        im2 = io.imread(os.path.join(images_path, image_name2))
        io.imsave(os.path.join(os.getcwd(), "Images_close_to_gt", image_name2), im2)
        
        # Get the ground truths out path
        gt_out_path = os.path.join(os.getcwd(), "GT_close_to_gt") 
        with open(os.path.join(os.getcwd(), "GT_close_to_gt", gt_name), "w") as f: 
            for el in deltas:
                p_orig = np.asarray([[el[0], el[1]]])
                # use the rotation matrix on the original point
                p_new = np.matmul(p_orig, rotate_matrix)[0, :2]
                # the gold standard point stays the same
                f.write(f"{round(p_new[0])} {round(p_new[1])} {el[2]} {el[3]}\n")  
                
    print("Image " +str(i)+" and its pair done.") 
    